In [25]:
!pip install pathway --quiet

import pandas as pd
import numpy as np
from math import radians, cos, sin, asin, sqrt
from datetime import datetime, timedelta
import pathway as pw


In [34]:
df = pd.read_csv('dataset.csv')

df['timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'], format='%d-%m-%Y %H:%M:%S')

df = df.drop(columns=['LastUpdatedDate', 'LastUpdatedTime'])
df = df.sort_values(by='timestamp')

In [40]:
!pip install pathway bokeh geopy

import numpy as np
import pandas as pd
from bokeh.plotting import figure, output_notebook, show
from bokeh.models import ColumnDataSource, Select
from bokeh.layouts import column, row
import pathway as pw
import datetime
from geopy.distance import geodesic

output_notebook()

unique_lots = df[['Latitude', 'Longitude']].drop_duplicates().reset_index(drop=True)
unique_lots['lot_id'] = unique_lots.index

# Create 'lot_id' in the main dataframe by mapping Latitude and Longitude
df = df.merge(unique_lots, on=['Latitude', 'Longitude'], how='left')

# Preview data
print("Dataset Preview:")
display(df.head())

# --- MODEL 1: BASELINE LINEAR MODEL ---

def baseline_linear_price(prev_price, occupancy, capacity, alpha=0.5):
    usage_ratio = occupancy / capacity
    return prev_price + alpha * usage_ratio

# --- MODEL 2: DEMAND-BASED MODEL ---

def demand_based_price(base_price, occupancy, capacity, queue, traffic, special_day, vehicle_type,
                        alpha=1.0, beta=0.5, gamma=0.3, delta=1.0, epsilon=0.4, vehicle_weights=None):
    if vehicle_weights is None:
        vehicle_weights = {'car': 1, 'bike': 0.6, 'cycle': 0.2,'truck': 1.5}

    vehicle_weight = vehicle_weights.get(vehicle_type, 1)
    demand = (alpha * (occupancy / capacity) +
              beta * queue -
              gamma * traffic +
              delta * special_day +
              epsilon * vehicle_weight)

    # Normalize demand between 0 and 1
    norm_demand = np.tanh(demand / 10)

    # Ensure price stays within 0.5x - 2x base
    price = base_price * (1 + 0.8 * norm_demand)
    return max(min(price, 2 * base_price), 0.5 * base_price)

# --- MODEL 3: COMPETITIVE PRICING MODEL ---

def competitive_price(lot_id, lots_df, current_lot_row, base_price, own_price, radius_km=1.0):
    current_coords = (current_lot_row['latitude'], current_lot_row['longitude'])

    competitors = []
    for _, row in lots_df.iterrows():
        if row['lot_id'] == lot_id:
            continue
        competitor_coords = (row['latitude'], row['longitude'])
        distance = geodesic(current_coords, competitor_coords).km
        if distance <= radius_km:
            competitors.append(row)

    # Simplified logic: average competitor price
    if not competitors:
        return own_price

    competitor_prices = [r['price'] for r in competitors]
    avg_comp_price = np.mean(competitor_prices)

    # If current lot is full and others are cheaper, reduce price or reroute
    if current_lot_row['occupancy'] >= current_lot_row['capacity'] and avg_comp_price < own_price:
        return max(own_price - 1, base_price * 0.5)  # reduce price but not too low

    # If nearby lots are more expensive
    elif avg_comp_price > own_price:
        return min(own_price + 1, base_price * 2.0)

    return own_price

# --- REAL-TIME SIMULATION SETUP (PATHWAY) ---

def simulate_realtime(df):
    for _, row in df.iterrows():
        yield row

# --- VISUALIZATION SETUP (BOKEH) FOR MULTIPLE LOTS WITH COMPETITOR COMPARISON ---

lot_ids = df['lot_id'].unique()
lot_sources = {lot: ColumnDataSource(data=dict(time=[], own_price=[], competitor_price=[])) for lot in lot_ids}

plots = []
for lot in lot_ids:
    p = figure(x_axis_type='datetime', title=f"Real-Time Pricing - Lot {lot}", width=800, height=300)
    p.line('time', 'own_price', source=lot_sources[lot], color='blue', legend_label=f"Lot {lot} Price")
    p.line('time', 'competitor_price', source=lot_sources[lot], color='red', legend_label="Avg Competitor Price", line_dash="dashed")
    p.legend.location = "top_left"
    plots.append(p)

# Simulate real-time pricing updates for each lot
base_price = 10.0
lot_prices = {lot: base_price for lot in lot_ids}

for t in range(10):  # Replace with real-time streaming logic
    ts = datetime.datetime.now() + datetime.timedelta(minutes=t*30)
    current_rows = []

    # Simulated state of each lot
    for lot in lot_ids:
        occupancy = 5 + (t % 5) * 2
        capacity = 20
        lat = df[df['lot_id'] == lot].iloc[0]['Latitude'] # Corrected column name to Latitude
        lon = df[df['lot_id'] == lot].iloc[0]['Longitude'] # Corrected column name to Longitude

        row_data = {
            'lot_id': lot,
            'latitude': lat,
            'longitude': lon,
            'occupancy': occupancy,
            'capacity': capacity,
            'price': lot_prices[lot],
        }
        current_rows.append(row_data)

    snapshot_df = pd.DataFrame(current_rows)

    for lot in lot_ids:
        lot_row = snapshot_df[snapshot_df['lot_id'] == lot].iloc[0]
        own_price = baseline_linear_price(lot_prices[lot], lot_row['occupancy'], lot_row['capacity'])
        final_price = competitive_price(lot, snapshot_df, lot_row, base_price, own_price)

        # Recalculate average competitor price for plot
        current_coords = (lot_row['latitude'], lot_row['longitude'])
        competitor_prices = []
        for _, r in snapshot_df.iterrows():
            if r['lot_id'] == lot:
                continue
            dist = geodesic(current_coords, (r['latitude'], r['longitude'])).km
            if dist <= 1.0:
                competitor_prices.append(r['price'])
        avg_comp_price = np.mean(competitor_prices) if competitor_prices else own_price

        lot_prices[lot] = final_price
        lot_sources[lot].stream({'time': [ts], 'own_price': [final_price], 'competitor_price': [avg_comp_price]}, rollover=100)

show(column(*plots))
print("Simulation Complete.")

Dataset Preview:


,ID,SystemCodeNumber,Capacity,Latitude,Longitude,Occupancy,VehicleType,TrafficConditionNearby,QueueLength,IsSpecialDay,LastUpdatedDate,LastUpdatedTime,lot_id
0,0,BHMBCCMKT01,577,26.144536,91.736172,61,car,low,1,0,04-10-2016,07:59:00,0
1,1,BHMBCCMKT01,577,26.144536,91.736172,64,car,low,1,0,04-10-2016,08:25:00,0
2,2,BHMBCCMKT01,577,26.144536,91.736172,80,car,low,2,0,04-10-2016,08:59:00,0
3,3,BHMBCCMKT01,577,26.144536,91.736172,107,car,low,2,0,04-10-2016,09:32:00,0
4,4,BHMBCCMKT01,577,26.144536,91.736172,150,bike,low,2,0,04-10-2016,09:59:00,0


Simulation Complete.
